<a href="https://colab.research.google.com/github/NNLP-IL/Hebrew-Question-Answering-Dataset/blob/main/Quality_control_for_Hebrew_question_answering_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Quality control for Hebrew question answering dataset




During the annotation process, we manually validated the data based on specific features of the question or the answer. This notebook extracts features to indicate instances where the same words appear in both the question and the answer, where no WH question word is used, where the answer occurs several times in the paragraph or when the answer span cuts a word in the middle.

import and download the data

In [ ]:
import numpy as np
import pandas as pd
import re
import difflib
from re import finditer

In [ ]:
import gdown
gdown.download(url='https://drive.google.com/file/d/1hPbyUSUyiv_WWMRqbmgxZHHcg9eux2nM/view?usp=sharing', output="sample.csv",quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1hPbyUSUyiv_WWMRqbmgxZHHcg9eux2nM
To: /content/sample.csv
100%|██████████| 12.9k/12.9k [00:00<00:00, 25.8MB/s]


'sample.csv'

In [ ]:
df = pd.read_csv('/content/sample.csv')

In [ ]:
df.head(10)

,prolificID,filename,title,context,question,answer_start,answer,answerable question,id_question
0,a,24,ג'וזי כץ,"בשנת 1972 הצטרפה כץ לצמד הדודאים, להרכב שנקרא ...",מתי הצטרפה כץ לצמד הדודאים?,0,בשנת 1972 הצטרפה כץ לצמד הדודאים,True,1
1,b,24,ג'וזי כץ,"בשנת 1972 הצטרפה כץ לצמד הדודאים, להרכב שנקרא ...",מתי הצטרפה כץ לדודאים,0,בשנת 1972,True,2
2,c,4139,זינדין זידאן,זידאן החל את הקריירה במועדון הכדורגל קאן. ב-19...,מתי זידאן נולד?,42,ב-1992,False,3
3,d,6404,ראש גדול (סדרת טלוויזיה),"איתי שותף בחברת פרסום יחד עם דניאל, יחד הם מנס...",עם מי דיתי הייתה אמורה להתחתן?,751,דניאל,True,4
4,e,266,ג'סטין לונג,"לונג נולד בפירפילד, קונטיקט שבארצות הברית, למש...",איפה נמצאת אוניברסיטת סקראד הארט?,11,פירפילד,True,5
5,f,266,ג'סטין לונג,"לונג נולד בפירפילד, קונטיקט שבארצות הברית, למש...",מי מביו אחיו של לונג עוסק בהוראה?,212,מיאן,True,6
6,c,5605,חטיבת כפיר,"תחת פיקוד חטיבת הצנחנים, גדוד חרוב הוביל את הל...",באיזו שנה הוקם גדוד חרוב?,176,2002,False,7
7,c,1752,פחמן דו-חמצני,"תופעות קרסט - המסת סלע גירי ויצירת מערות, בולע...",מהי תופעת קרסט?,14,"המסת סלע גירי ויצירת מערות, בולענים, נטיפים ות...",True,8
8,g,359,פדרה (סנקה),תמונה ב: פדרה כלאה את עצמה בחדר שלא תראה אף אח...,תיזאוס הוא בעלה של?,408,פדרה,True,9
9,h,4894,מחשב שלם בתוך מקלדת: בדקנו את Raspberry Pi 400,"בעבודה השוטפת, לא נתקלתי בבעיות מיוחדות בתפעול...",זום עובד על המחשב?,529,תצטרכו להשתמש בגרסת הווב,True,10


##Functions

In [ ]:
def remove_niqqud_from_string(my_string):
    """Get a Hebrew string and replace punctuation marks with space ('').

    Parameters
    ----------
    my_string : str
        Hebrew sentence

    Returns
    -------
    string
        The same input -Hebrew sentence but without punctuation.
    """
    return ''.join(['' if  1456 <= ord(c) <= 1479 else c for c in my_string])  

In [ ]:
def high_light_question(row):
    """Highlight the answer within the paragraph.

    Parameters
    ----------
    row : series
        One row of the dataframe that includes at least the following columns:
        "context", "answer", "answer_start

    Returns
    -------
    str
        the paragraph with "*" marks that shows the answer's location.
    """
    start = row['answer_start']
    end = start + len(row['answer'])
    return row['context'][:start] + '*' + row['context'][start:end] + '*' + row['context'][end:]

In [ ]:
def how_many_contain_string(row, squad=True):
    """Checks the overlap between the question and the answer
    - the number of overlapping words.

    Parameters
    ----------
    row : series
        One row of the dataframe that includes at least the following columns:
        "question","answer"
    squad : bool, default True.
        If the dataset is in Hebrew, set the argument to true to delete punctuation marks.

    Returns
    -------
    int
        the number of overlapping words.
    """
    if pd.isnull(row['answer']):
        return np.nan

    if squad:
        context = remove_niqqud_from_string(row['answer'])
        question = remove_niqqud_from_string(row['question'])
    else:
        context = row['answer']
        question = row['question']
    
    context = set(re.sub(r'[^\w\s]',' ',context).split(' '))
    question = set(re.sub(r'[^\w\s]',' ',question).split(' '))
    temp = [i for i in question.intersection(context) if len(i) > 0]
    return len(temp)

In [ ]:
def wh_q_list_fun(wh_q, prefix, more_wh_q_with_prefix): 
    """Returns every combination of question word and prefix (בכמ"ל).

    Parameters
    ----------
    wh_q : list
        List of question words
    prefix : list
        List of prefixes
    more_wh_q_with_prefix : list
        An additional list of combinations between question words and prefixes.

    Returns
    -------
    list
        Returns the combinations between the question words and prefixes.
    """
    combo_wh = [[x,y] for x in prefix for y in wh_q] 
    wh_q_with_prefix = [i[0] + i[1] for i in combo_wh]
    return wh_q_with_prefix + more_wh_q_with_prefix

In [ ]:
def get_overlap_chr(s1, s2):
    """Returns the longest overlap (at character level) between two strings.

    Parameters
    ----------
    s1 : str
        The first question word.
    s2 : str
        The second question word.

    Returns
    -------
    str
        The overlapping part
    """
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return s1[pos_a:pos_a+size]

In [ ]:
def wh_detect(row) ->str : 
    """
    Adds two features:
    WH Question Words - all the question words that were used in the "question" field, exactly as they appeared, including prepositions (for example: from where, in which)
    WH Question Original - the question word used, without prepositions (from where becomes where and in which becomes which)

    Parameters
    ----------
    row : series
        One row of the dataframe that includes at least the following column:
        "question".

    Returns
    -------
    series
        The same input series include adding the two new columns of the WH Question words.
    """
    qus = row['question']
    qus = re.sub(r'[^\w\s]','',qus)
    qus = qus.split(' ')
    loc_q = [x in wh_q_list for x in qus]
    if any(loc_q):
        qus_final = qus[np.where(loc_q)[0][0]]
        row['WH Question Words'] = qus_final
        # special cases
        if qus_final == 'מזה':
            row['WH Question Original'] = 'מה'
            return row
        if qus_final in ['מיהי','מיהו','מיהם','מיהן']:
            row['WH Question Original'] = 'מי'
            return row
  
        # Find the mother of the word
        temp = [len(get_overlap_chr(qus_final,wh)) for wh in wh_q]
        row['WH Question Original'] = wh_q[temp.index(max(temp))]
        if (len(np.where(loc_q)[0]) > 1) & ('האם' not in qus):
            qus_final_a = qus[np.where(loc_q)[0][0]]
            qus_final_b = qus[np.where(loc_q)[0][1]]
            row['WH Question Words'] = qus_final_a + ' ' + qus_final_b
    else:
        row['WH Question Words'] = False
        row['WH Question Original'] = False
    return row

In [ ]:
def answer_cut_word(row):
    """Check if the answer span cuts a word in the middle.
    Answers that are cut off after the letters מש"ה וכל"ב are not counted (which, in this case, is correct). 

    Parameters
    ----------
    row : series
        One row of the dataframe that includes at least the following column:
        "context"

    Returns
    -------
    int
        Number that indicating how many words have been cut.
    """
    temp = row['context'].replace('"','').replace(':','').replace('”','').replace('“','').replace('.',' ').replace(',',' ').replace('–',' ').replace(';',' ').replace('-',' ').replace('־',' ').replace(')',' ').replace('(',' ').replace("'",' ') # ignore if " before or after the answer

    temp = temp.split()
    
    temp = [i[1:] if i[:1] in 'משהוכלב' else i for i in temp] # ignore if משה וכלב before
    temp = [i[1:] if i[:1] in 'משהוכלב' else i for i in temp] # ignore if ו before
    temp = [i[1:] if i[:1] in 'משהוכלב' else i for i in temp] # ignore if ו before

    temp = [True for i in temp if i[1:-1].find('*') != -1]
    
    return len(temp)

In [ ]:
def par_repeat(row):
    """Returns True if the answer appears more than once in a paragraph.

    Parameters
    ----------
    row : series
        One row of the dataframe that includes at least the following columns:
        "answer","context".

    Returns
    -------
    bool
        Returns true if the answer appears more than once in the paragraph. False otherwise.
    """
    ans = row['answer']
    ans = re.sub(r'[^\w\s]','',ans)
    con = row['context']
    con = re.sub(r'[^\w\s]','',con)
    flag = [match.start(0) for match in finditer(ans, con)]
    if len(flag) > 1:
        return True
    else:
        return False

##Question words


Define Hebrew question words and combinations of question words and prefixes.

In [ ]:
wh_q = ['אלו','מה','מי','למה', 'מדוע' ,'איפה', 'היכן', 'איה','לאן','מאין','מתי','איך', 'כיצד','כמה','האם','איזה','מדוע','מנין','איזו','אילו','הגם', 'האומנם','איזו']
prefix = ['כ','מ','ל','ב','','ש']
more_wh_q_with_prefix = ['מהו','מיהי','מהי','מיהו','מהן','מהם','מיהם','מזה','מיהן']

In [ ]:
wh_q_list = wh_q_list_fun(wh_q, prefix, more_wh_q_with_prefix)

##Feature engineering for quality measures

In [ ]:
# Mark the answer in the paragraph
df['context_with_marked_answer'] = df.apply(high_light_question, axis=1)

In [ ]:
# Counting the overlapping words between the question and the answer.
df['overlap_q&answer_tokens'] = df.apply(how_many_contain_string, axis=1)

In [ ]:
# wh question
df = df.apply(wh_detect, axis=1)

In [ ]:
# text lengths
df['context_len'] = df['context'].str.len()
df['answer_len'] = df['answer'].str.len()
df['question_len'] = df['question'].str.len()

In [ ]:
#if the answer appears more than once in a paragraph
df['answer_occurs_twice_or_more'] = df.apply(par_repeat, axis=1)

In [ ]:
# Check if the beginning of the answer, the word is cut off in the middle.
df['answer_cut_word'] = df.apply(answer_cut_word, axis=1)

## Output

In [ ]:
df.head(10)

,prolificID,filename,title,context,question,answer_start,answer,answerable question,id_question,context_with_marked_answer,overlap_q&answer_tokens,WH Question Words,WH Question Original,context_len,answer_len,question_len,answer_occurs_twice_or_more,answer_cut_word
0,a,24,ג'וזי כץ,"בשנת 1972 הצטרפה כץ לצמד הדודאים, להרכב שנקרא ...",מתי הצטרפה כץ לצמד הדודאים?,0,בשנת 1972 הצטרפה כץ לצמד הדודאים,True,1,"*בשנת 1972 הצטרפה כץ לצמד הדודאים*, להרכב שנקר...",4,מתי,מתי,725,32,27,False,0
1,b,24,ג'וזי כץ,"בשנת 1972 הצטרפה כץ לצמד הדודאים, להרכב שנקרא ...",מתי הצטרפה כץ לדודאים,0,בשנת 1972,True,2,"*בשנת 1972* הצטרפה כץ לצמד הדודאים, להרכב שנקר...",0,מתי,מתי,725,9,21,False,0
2,c,4139,זינדין זידאן,זידאן החל את הקריירה במועדון הכדורגל קאן. ב-19...,מתי זידאן נולד?,42,ב-1992,False,3,זידאן החל את הקריירה במועדון הכדורגל קאן. *ב-1...,0,מתי,מתי,561,6,15,False,0
3,d,6404,ראש גדול (סדרת טלוויזיה),"איתי שותף בחברת פרסום יחד עם דניאל, יחד הם מנס...",עם מי דיתי הייתה אמורה להתחתן?,751,דניאל,True,4,"איתי שותף בחברת פרסום יחד עם דניאל, יחד הם מנס...",0,מי,מי,847,5,30,True,0
4,e,266,ג'סטין לונג,"לונג נולד בפירפילד, קונטיקט שבארצות הברית, למש...",איפה נמצאת אוניברסיטת סקראד הארט?,11,פירפילד,True,5,"לונג נולד ב*פירפילד*, קונטיקט שבארצות הברית, ל...",0,איפה,איפה,706,7,33,True,0
5,f,266,ג'סטין לונג,"לונג נולד בפירפילד, קונטיקט שבארצות הברית, למש...",מי מביו אחיו של לונג עוסק בהוראה?,212,מיאן,True,6,"לונג נולד בפירפילד, קונטיקט שבארצות הברית, למש...",0,מי,מי,706,4,33,False,0
6,c,5605,חטיבת כפיר,"תחת פיקוד חטיבת הצנחנים, גדוד חרוב הוביל את הל...",באיזו שנה הוקם גדוד חרוב?,176,2002,False,7,"תחת פיקוד חטיבת הצנחנים, גדוד חרוב הוביל את הל...",0,באיזו,איזו,577,4,25,True,0
7,c,1752,פחמן דו-חמצני,"תופעות קרסט - המסת סלע גירי ויצירת מערות, בולע...",מהי תופעת קרסט?,14,"המסת סלע גירי ויצירת מערות, בולענים, נטיפים ות...",True,8,"תופעות קרסט - *המסת סלע גירי ויצירת מערות, בול...",0,מהי,מה,512,62,15,False,0
8,g,359,פדרה (סנקה),תמונה ב: פדרה כלאה את עצמה בחדר שלא תראה אף אח...,תיזאוס הוא בעלה של?,408,פדרה,True,9,תמונה ב: פדרה כלאה את עצמה בחדר שלא תראה אף אח...,0,False,False,529,4,19,True,0
9,h,4894,מחשב שלם בתוך מקלדת: בדקנו את Raspberry Pi 400,"בעבודה השוטפת, לא נתקלתי בבעיות מיוחדות בתפעול...",זום עובד על המחשב?,529,תצטרכו להשתמש בגרסת הווב,True,10,"בעבודה השוטפת, לא נתקלתי בבעיות מיוחדות בתפעול...",0,False,False,607,24,18,False,0


*   The “**overlap_q&answer_tokens**” feature indicated cases where identical words 
appear in both the question and the answer. The correct answer span should only include the minimal answer and not the entire sentence where it appears. Therefore, the lexical overlap between a question and its corresponding answer is usually minimal. Generally, words from the question will not appear in the answer, and if there is an overlap, it typically consists of only one or two words, often a stop word like “של“. This feature can help us locate the incorrect answer span for question number 1. An example for a correct answer span is demonstrated in question number 2.
*   The “**WH Question**” features indicate cases where no WH question word is used. Although this often indicates a typo in the question wording, the main purpose was to make sure that the question was not phrased as a sentence completion task (question 9), a yes-no question (question 10).
*   The “**answer_occurs_twice_or_more**” feature indicates cases where the answer occurs several times in the paragraph. We applied this feature to ensure that the answer was correctly identified in the appropriate location. For example, in question 5, the answer was located in an incorrect spot (“לונג נולד ב*פירפילד*“), and requires modification to reflect the correct location (“באוניברסיטת סקראד הארט שב*פירפילד*“). The “**context_with_marked_answer**” feature enabled us to review the answer's location.
*   The “**answer_cut_word**” feature indicates cases where the answer span cuts a word in the middle (question 6), unless the word is cut after prefix (מש״ה וכל״ב). 